In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
# Data augmentation for training
train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2,
    saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
    [0.229, 0.224, 0.225])
])

# No augmentation for validation/test
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
    [0.229, 0.224, 0.225])
])

In [ ]:
# Load datasets
train_dataset = datasets.ImageFolder('data/train', transform=train_transforms)
val_dataset = datasets.ImageFolder('data/val', transform=val_transforms)
test_dataset = datasets.ImageFolder('data/test', transform=val_transforms)

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f'Training samples: {len(train_dataset)}')
print(f'Validation samples: {len(val_dataset)}')
print(f'Test samples: {len(test_dataset)}')

Training samples: 20000
Validation samples: 2500
Test samples: 2459


In [ ]:
 # Load pretrained ResNet18
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 109MB/s]


In [ ]:
# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# Replace final layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

In [ ]:
# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f'Using device: {device}')
print(f'Training only final layer with {model.fc.in_features} output to 2')

Using device: cuda
Training only final layer with 512 output to 2


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [ ]:
# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

In [ ]:
# Training tracking
train_losses = []
val_losses = []
train_accs = []
val_accs = []
best_val_acc = 0.0

num_epochs = 10

In [ ]:
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_running_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    # Update learning rate
    scheduler.step(val_loss)

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'Saved best model with val_acc: {val_acc:.2f}%')

    print(f'Epoch [{epoch+1}/{num_epochs}]')
    print(f' Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f' Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print()


Saved best model with val_acc: 97.80%
Epoch [1/10]
 Train Loss: 0.1590, Train Acc: 93.71%
 Val Loss: 0.0601, Val Acc: 97.80%

Saved best model with val_acc: 98.00%
Epoch [2/10]
 Train Loss: 0.1162, Train Acc: 95.24%
 Val Loss: 0.0543, Val Acc: 98.00%

Saved best model with val_acc: 98.08%
Epoch [3/10]
 Train Loss: 0.1170, Train Acc: 95.25%
 Val Loss: 0.0518, Val Acc: 98.08%

Epoch [4/10]
 Train Loss: 0.1090, Train Acc: 95.47%
 Val Loss: 0.0519, Val Acc: 98.08%

Epoch [5/10]
 Train Loss: 0.1118, Train Acc: 95.44%
 Val Loss: 0.0508, Val Acc: 98.00%

Saved best model with val_acc: 98.40%
Epoch [6/10]
 Train Loss: 0.1067, Train Acc: 95.59%
 Val Loss: 0.0445, Val Acc: 98.40%

Epoch [7/10]
 Train Loss: 0.1102, Train Acc: 95.70%
 Val Loss: 0.0429, Val Acc: 98.32%

Epoch [8/10]
 Train Loss: 0.1089, Train Acc: 95.64%
 Val Loss: 0.0586, Val Acc: 98.00%

Epoch [9/10]
 Train Loss: 0.1087, Train Acc: 95.53%
 Val Loss: 0.0472, Val Acc: 98.16%

Saved best model with val_acc: 98.72%
Epoch [10/10]
 Tra